In [1]:
import pandas as pd
import folium
from folium import IFrame

In [2]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("Søre Goppollvegen,  2634 Fåvang", exactly_one=True)
print(location)
location = geolocator.geocode("2634 Fåvang", exactly_one=True)
print(location)

None
Fåvang, Ringebu, Oppland, 2634, Norge


In [3]:
df = pd.read_excel('LeveranseTilSlakteriLandbruket_Alle_Storfe.xlsx')

In [4]:
df.columns

Index(['orgnr', 'navn', 'komnr', 'kalv_kg', 'ku_kg', 'kvige_kg', 'okse_kg',
       'ungokse_kastrat_kg', 'ungku_kg', 'år', 'komnavn', 'organisasjonsform',
       'forretningsadr', 'forradrpostnr', 'forradrpoststed', 'slaktsum_kg'],
      dtype='object')

In [5]:
df.shape

(46225, 16)

In [6]:
same_headers = ['navn','år','orgnr','komnr','komnavn','organisasjonsform','forretningsadr', 'forradrpostnr', 'forradrpoststed']
df2013 = df[df['år']==2013][df.columns.difference(same_headers)].add_suffix('_2013')
df2013[same_headers] = df[df['år']==2013][same_headers]
df2014 = df[df['år']==2014][df.columns.difference(same_headers)].add_suffix('_2014')
df2014[same_headers] = df[df['år']==2014][same_headers]
df2015 = df[df['år']==2015][df.columns.difference(same_headers)].add_suffix('_2015')
df2015[same_headers] = df[df['år']==2015][same_headers]
df2016 = df[df['år']==2016][df.columns.difference(same_headers)].add_suffix('_2016')
df2016[same_headers] = df[df['år']==2016][same_headers]

In [15]:
print(df2013.shape)
print(df2014.shape)
print(df2015.shape)
print(df2016.shape)

(11484, 16)
(11236, 16)
(11788, 16)
(11717, 16)


In [23]:
df2016['address'] = df2016['forretningsadr'] + ", "+ df2016['forradrpostnr'].map(int).map(str)+ " " + df2016['forradrpoststed']

In [25]:
df2016['address'].head()

34508                           Hestadveien 159, 4588 KVÅS
34509               Hamnnesvegen 44, 6443 TORNES I ROMSDAL
34510    v/Svein Olav Tengesdal Åseveien 60, 4372 EGERSUND
34511                                   Hoel, 7393 RENNEBU
34512                    Strompdalsveien 1076, 7892 TRONES
Name: address, dtype: object

In [26]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

In [27]:
def get_lat_lon(r):
    print(r,end='\r')
    try:
        return geolocator.geocode(r)[-1]
    except:
        return (float('nan'),float('nan'))
        

In [28]:
df2016.shape

(11717, 17)

In [29]:
df2 = df2016.head(400)

In [30]:
df2.loc[:,('lat')], df2.loc[:,('lon')] = zip(*df2.loc[:,('address')].map(get_lat_lon))

/home/patechoc/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/patechoc/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [31]:
df2.columns

Index(['kalv_kg_2016', 'ku_kg_2016', 'kvige_kg_2016', 'okse_kg_2016',
       'slaktsum_kg_2016', 'ungku_kg_2016', 'ungokse_kastrat_kg_2016', 'navn',
       'år', 'orgnr', 'komnr', 'komnavn', 'organisasjonsform',
       'forretningsadr', 'forradrpostnr', 'forradrpoststed', 'address', 'lat',
       'lon'],
      dtype='object')

In [32]:
df2[['lat','lon']].head()

,lat,lon
34508,NaN,NaN
34509,NaN,NaN
34510,NaN,NaN
34511,62.853549,9.83224
34512,NaN,NaN


In [33]:
df3  = df2[df2['lat'].notnull()]
df3.head()
df3.shape

(114, 19)

In [38]:
# df3.to_csv('farmers_dataFrame_2016.csv')
# df3.to_excel('farmers_dataFrame_2016.xlsx')

In [39]:
df3 = pd.read_csv('farmers_dataFrame_2016.csv')
# df3['år'].unique()
df3.shape

(295, 21)

### 

In [50]:
import random

df3 = pd.read_csv('farmers_dataFrame.csv')

data = df3[['lat', 'lon','navn', 'slaktsum_kg','år','orgnr']]

# Make an empty map
# my_map = folium.Map(location=[60.4720, 8.4689], tiles="Mapbox Bright", zoom_start=2)
my_map = folium.Map(location=[60.4720, 8.4689], tiles="Stamen Terrain", zoom_start=5)


folium.TileLayer('openstreetmap').add_to(my_map)
# other mapping code (e.g. lines, markers etc.)
folium.LayerControl().add_to(my_map)

for i in range(0,len(data)):
    orgnr=data.iloc[i]['orgnr']
    try:
        slg2013=str(int(df2013[df2013['orgnr']==orgnr]['slaktsum_kg_2013']))
        slg2014=str(int(df2014[df2014['orgnr']==orgnr]['slaktsum_kg_2014']))
        slg2015=str(int(df2015[df2015['orgnr']==orgnr]['slaktsum_kg_2015']))
        slg2016=str(int(df2016[df2016['orgnr']==orgnr]['slaktsum_kg_2016']))
    except:
        pass
    html = """<head>
</head><h2 style="text-align: center;"><span style="color: #3366ff;">{farmerName}</span></h2>
<table align="center" style="font-size:10px">
<tbody>	
<tr>
<td><span style="color: #808080;"><strong>Org. nr.:</strong></span></td>
<td><strong>{orgnr}</strong></td>
</tr>
<tr>
<td><span style="color: #808080;"><strong># <span id="result_box" class="short_text" lang="no" tabindex="-1"><span class="">Totalt antall kyr:</span></span></strong></span></td>
<td><strong>{iDagKyr}</strong></td>
</tr>
<tr>
<td><span style="color: #808080;"><strong><span id="result_box" class="short_text" lang="no" tabindex="-1">
<span class=""># slaktet i 2013</span></span></strong></span></td>
<td><strong><span id="result_box" class="short_text" lang="no" tabindex="-1"><span class="">{slg2013}</span>
</span></strong></td>
</tr>
<tr>
<td><span style="color: #808080;"><strong><span id="result_box" class="short_text" lang="no" tabindex="-1"><span class=""># slaktet i 2014</span></span></strong></span></td>
<td><strong><span id="result_box" class="short_text" lang="no" tabindex="-1"><span class="">{slg2014}</span></span></strong></td>
</tr>
<tr>
<td><span style="color: #808080;"><strong><span id="result_box" class="short_text" lang="no" tabindex="-1"></span><span id="result_box" class="short_text" lang="no" tabindex="-1"><span class=""># slaktet i 2015</span></span></strong></span></td>
<td><strong><span id="result_box" class="short_text" lang="no" tabindex="-1"><span class="">{slg2015}</span></span></strong></td>
</tr>
<tr>
<td><span style="color: #808080;"><strong><span id="result_box" class="short_text" lang="no" tabindex="-1"><span class=""># slaktet i 2016</span></span></strong></span></td>
<td><strong>{slg2016}</strong></td>
</tr>
</tbody>
</table>
<p style="text-align: center;"><button style="background-color: #4CAF50;    border: none;
    color: white;
    padding: 15px 32px;
    text-align: center;
    text-decoration: none;
    display: inline-block;
    font-size: 16px;
    margin: 4px 2px;
    cursor: pointer;" class="button" onClick="window.open('{url}');"><span class="icon">{nameButton}</span></button></p>
    """.format(orgnr=data.iloc[i]['orgnr'],
               farmerName=data.iloc[i]['navn'].title(), 
               iDagKyr=str(int(data.iloc[i]['slaktsum_kg']/(3*300))),
               slg2013=slg2013,
               slg2014=slg2014,
               slg2015=slg2015,
               slg2016=slg2016,
               url='http://www.google.com',
               nameButton='Gå til rapporten')
    iframe = IFrame(html=html, width=250, height=250)
    popup = folium.Popup(iframe, max_width=400)
    r = random.random()
    if r > 0.7:
        color = 'green'
    elif r > 0.3:
        color = "orange"
    else:
        color = 'red'
    folium.Marker([data.iloc[i]['lat'], data.iloc[i]['lon']], popup=popup, icon=folium.Icon(color=color)).add_to(my_map)
#     folium.Marker([data.iloc[i]['lat'], data.iloc[i]['lon']], popup=data.iloc[i]['navn']).add_to(my_map)

my_map.save('producerMap.html')
my_map